# Counterfactual Explanations:

In this notebook, we will explore counterfactual explanations, which are post-hoc explainability methods specifying parts of an image .

In [ ]:
import os
import re
from glob import glob
import torch
import random
import numpy as np
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
from imgaug import augmenters as iaa
from sklearn.preprocessing import LabelEncoder
import cv2
from tqdm import tqdm

from torch.utils.data.sampler import Sampler

import torch
from torch.nn import functional as F
from torch.nn.modules import Module
# from utils.utils import get_roc_auc_score
# from utils.utils import prototype_heatmap
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import roc_auc_score, roc_curve, f1_score, recall_score, precision_score
from torchvision import transforms
import sys
import argparse
import logging
import random
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch import optim
from torch.utils.data import DataLoader, Subset
from torchvision import transforms
from datetime import datetime
from model_intepretability.imaging_copy.dataset.dataset import PrototypicalBatchSampler
from sklearn.metrics import roc_auc_score, roc_curve
from torch.nn.parallel import DistributedDataParallel as DDP
from functools import partial
import torch.distributed as dist
from sklearn.metrics import roc_auc_score, roc_curve, f1_score, recall_score, precision_score
from torch.utils.data import DataLoader,SubsetRandomSampler, TensorDataset
from sklearn.model_selection import KFold
from torch.utils.data.distributed import DistributedSampler
import torch.nn.functional as F
import pickle
import torchvision.models as models
import torch.distributed as dist
from tqdm import tqdm
from torch.autograd import Variable
plt.rcParams['figure.figsize'] = [25, 10]
import gradio as gr
from glob import glob
from PIL import Image
import argparse
import gc
from pathlib import Path
# from utils.utils import get_roc_auc_score
from omnixai.data.image import Image
from omnixai.explainers.vision import CounterfactualExplainer

In [ ]:
def gather(tensor, tensor_list=None, root=0, group=None):
    """
        Sends tensor to root process, which store it in tensor_list.
    """

    rank = dist.get_rank()
    if group is None:
        group = dist.group.WORLD
    if rank == root:
        assert(tensor_list is not None)
        dist.gather(tensor, gather_list=tensor_list, group=group)
    else:
        dist.gather(tensor, dst=root, group=group)



def compute_test_auc(model, test_loader, device_id):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.cuda(device_id, non_blocking=True), labels.cuda(device_id, non_blocking=True)
            outputs,_ = model(images)
            preds = torch.sigmoid(outputs)#.squeeze()  # Assuming binary classification

            # Gather predictions and labels across all processes

            gathered_preds = [torch.zeros_like(preds) for _ in range(dist.get_world_size())]
            gathered_labels = [torch.zeros_like(labels) for _ in range(dist.get_world_size())]

            # Collect predictions and labels from each process

            dist.all_gather(gathered_preds,preds)
            dist.all_gather(gathered_labels,labels)


            # Only concatenate and store predictions and labels on rank 0
            if dist.get_rank() == 0:
                all_preds.extend([p.cpu() for p in gathered_preds])
                all_labels.extend([l.cpu() for l in gathered_labels])


    # On rank 0, compute AUC
    if dist.get_rank() == 0:
        all_preds = torch.cat(all_preds)
        all_labels = torch.cat(all_labels)
        print("lllll",all_labels.shape)
        print("sssss",all_preds.shape)
        auc = get_roc_auc_score(all_labels.numpy(), all_preds.numpy())
        print(f"AUC: {auc}")
        return auc

def aggregate_all_predictions(predictions, labels,device_id):


    # Initialize lists to gather predictions and labels from all devices

    predictions = torch.tensor(predictions).cuda(device_id)
    labels = torch.tensor(labels).cuda(device_id)

    gathered_preds = [torch.zeros_like(predictions) for _ in range(dist.get_world_size())]
    gathered_labels = [torch.zeros_like(labels) for _ in range(dist.get_world_size())]

    # Use all_gather to collect all data across devices
    dist.all_gather(gathered_preds, predictions)
    dist.all_gather(gathered_labels, labels)

    # Concatenate data across devices
    all_preds = torch.cat(gathered_preds)
    all_labels =torch.cat(gathered_labels)

    # Convert to numpy for AUC calculation
    return all_preds.cpu().numpy(), all_labels.cpu().numpy()

def load_checkpoint(model, optimizer,file_path):
    checkpoint = torch.load(file_path)

    checkpoint_state_dict = checkpoint['model_state_dict']


    # Create a new state_dict with only the matching keys
    checkpoint_state_dict = {k.replace('module.', ''): v for k, v in checkpoint_state_dict.items()}

    filtered_state_dict = {k: v for k, v in checkpoint_state_dict.items() if k in model.state_dict()}# and "mlp.mlp_layers.dense_1" not in k and "mlp.mlp_layers.dense_0" not in k and "num_norm" not in k}

    model.load_state_dict(filtered_state_dict, strict=False)

    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    return model, optimizer, epoch, loss

def worker_init_fn(worker_id: int, num_workers: int, rank: int, seed: int) -> None:
    """Initialize worker processes with a random seed.

    Parameters
    ----------
    worker_id : int
        ID of the worker process.
    num_workers : int
        Total number of workers that will be initialized.
    rank : int
        The rank of the current process.
    seed : int
        A random seed used determine the worker seed.
    """
    worker_seed = num_workers * rank + worker_id + seed
    torch.manual_seed(worker_seed)
    random.seed(worker_seed)

def save_checkpoint(model, optimizer, epoch, loss,file_path):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }
    torch.save(checkpoint, file_path)

def setup() -> None:
    """Initialize the process group."""
    dist.init_process_group("nccl")#,rank=dist.get_rank(), init_method='env://')


def cleanup() -> None:
    """Clean up the process group after training."""
    dist.destroy_process_group()

In [ ]:

num_workers=4
resize=224
batch_size = 32
epochs = 20
lr = 1e-3
scheduler = False
step_size = 5

dropout = 0.5
rseed = 42
weight_decay = 1e-3
num_epochs = 20
its = 100
cTr = 15
nsTr = 5
nqTr = 5
cVa = 5
nsVa = 5
nqVa = 15


## Data:

In this project, we use the "NIH" dataset, which is publicly accessible at [here](https://www.kaggle.com/datasets/nih-chest-xrays/data). It contains 112120 chest X-rays corresponding to 14 different conditions, e.g., Pneumonia. The goal is to train a predictive model to classify each image to one or multiple conditions.

In [ ]:
def read_image(image_path):
    image = cv2.imread(image_path)
    image = image/np.max(image)
    return image



class XrayDataset(Dataset):
    def __init__(self, csv_file, image_path_name):

        self.path_name = image_path_name
        self.csv_file = csv_file
        self.the_chosen, self.all_classes, self.all_classes_dict = self.choose_the_indices()
        self.csv_file["numeric_targets"] = self.csv_file['Finding Labels'].apply(lambda x: self.get_tagets(x))
    def __len__(self):

        return len(self.csv_file)
    def choose_the_indices(self):

        max_examples_per_class = 10000 # its the maximum number of examples that would be sampled in the training set for any class
        the_chosen = []
        all_classes = {}
        length = len(self.csv_file)
        print('\nSampling the huuuge training dataset')
        for i in tqdm(list(np.random.choice(range(length),length, replace = False))):

            temp = str.split(self.csv_file.iloc[i, :]['Finding Labels'], '|')

            # special case of ultra minority hernia. we will use all the images with 'Hernia' tagged in them.
            if 'Hernia' in temp:
                the_chosen.append(i)
                for t in temp:
                    if t not in all_classes:
                        all_classes[t] = 1
                    else:
                        all_classes[t] += 1
                continue

            # choose if multiple labels
            if len(temp) > 1:
                bool_lis = [False]*len(temp)
                # check if any label crosses the upper limit
                for idx, t in enumerate(temp):
                    if t in all_classes:
                        if all_classes[t]< max_examples_per_class: # 500
                            bool_lis[idx] = True
                    else:
                        bool_lis[idx] = True
                # if all lables under upper limit, append
                if sum(bool_lis) == len(temp):
                    the_chosen.append(i)
                    # maintain count
                    for t in temp:
                        if t not in all_classes:
                            all_classes[t] = 1
                        else:
                            all_classes[t] += 1
            else:        # these are single label images
                for t in temp:
                    if t not in all_classes:
                        all_classes[t] = 1
                    else:
                        if all_classes[t] < max_examples_per_class: # 500
                            all_classes[t] += 1
                            the_chosen.append(i)



        '''
        if len(the_chosen) != len(set(the_chosen)):
            print('\nGadbad !!!')
            print('and the difference is: ', len(the_chosen) - len(set(the_chosen)))
        else:
            print('\nGood')
        '''
        with open('all_classes.pkl', 'wb') as file:
            pickle.dump(all_classes, file)
        return the_chosen, sorted(list(all_classes)), all_classes

    def get_tagets(self,row):
        labels = str.split(row, '|')

        target = torch.zeros(len(self.all_classes))
        for lab in labels:
            lab_idx = self.all_classes.index(lab)
            target[lab_idx] = 1
        return target
    def get_image(self, idx):
        # -- Query the index location of the required file

        image_name = self.csv_file.loc[idx,'Image Index']

        image_path = glob(os.path.join(self.path_name, '**', image_name), recursive=True)[0]
        image = read_image(image_path)
        if len(image.shape) == 2: image = np.expand_dims(image, axis=-1)

        labels = str.split(self.csv_file.loc[idx,'Finding Labels'], '|')

        target = torch.zeros(len(self.all_classes))
        for lab in labels:
            lab_idx = self.all_classes.index(lab)
            target[lab_idx] = 1
        mean = [0.485, 0.456, 0.406]
        std = [0.229, 0.224, 0.225]
        input_size = 224
        rseed = 42
        seq = iaa.Sequential([iaa.Resize((input_size, input_size))])
        image_transform = transforms.Compose([seq.augment_image, transforms.ToTensor(), transforms.Normalize(mean=mean, std=std)])
        image = image_transform(image)
        return image.float(), target


    def num_sort(self, filename):
        not_num = re.compile("\D")
        return int(not_num.sub("", filename))

    def __getitem__(self, idx):
        image, label = self.get_image(idx)
        return image,label


In [ ]:
# model architechture
class SelfAttention(nn.Module):
    def __init__(self, in_dim):
        super(SelfAttention, self).__init__()
        self.query = nn.Conv2d(in_dim, in_dim // 8, kernel_size=1)
        self.key = nn.Conv2d(in_dim, in_dim // 8, kernel_size=1)
        self.value = nn.Conv2d(in_dim, in_dim, kernel_size=1)
        self.gamma = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        batch_size, C, width, height = x.size()
        query = self.query(x).view(batch_size, -1, width * height).permute(0, 2, 1)
        key = self.key(x).view(batch_size, -1, width * height)
        # calculating attention weights
        attention = torch.bmm(query, key)
        attention = F.softmax(attention, dim=-1)
        value = self.value(x).view(batch_size, -1, width * height)
        # calculating weighted context
        out = torch.bmm(value, attention.permute(0, 2, 1))
        out = out.view(batch_size, C, width, height)
        out = self.gamma * out + x # weighted context added to the original input in a residual manner
        return out, attention

class ResNetAttention(nn.Module):
    def __init__(self, original_model,num_classes=15):
        super(ResNetAttention, self).__init__()
        self.features = nn.Sequential(*list(original_model.children())[:-2])
        self.attention = SelfAttention(in_dim=512)  # Adjust channels based on ResNet block
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(512, 256)  # Output channels should match in_channels of attention layer
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.features(x)
        x, attention = self.attention(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x, attention

In [ ]:
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import roc_auc_score, roc_curve, f1_score, recall_score, precision_score
import numpy as np
import torch.nn.functional as F
import torch
from torchvision import transforms



def get_roc_auc_score(y_true, y_probs):
    '''
    Uses roc_auc_score function from sklearn.metrics to calculate the micro ROC AUC score for a given y_true and y_probs.
    '''

    with open('all_classes.pkl', 'rb') as all_classes:
        all_classes = pickle.load(all_classes)

    NoFindingIndex = all_classes.get('No Finding', -1)
    class_roc_auc_list = []
    useful_classes_roc_auc_list = []

    for i in range(y_true.shape[1]):
        if len(np.unique(y_true[:, i])) > 1:
            class_roc_auc = roc_auc_score(y_true[:, i], y_probs[:, i])
            class_roc_auc_list.append(class_roc_auc)
            if i != NoFindingIndex:
                useful_classes_roc_auc_list.append(class_roc_auc)
    return np.mean(np.array(useful_classes_roc_auc_list))



In [ ]:

auc_list , precision_list, recall_list, f1_list = [], [], [], []
random.seed(rseed)
np.random.seed(rseed)
torch.manual_seed(rseed)
torch.cuda.manual_seed(rseed)
torch.cuda.manual_seed_all(rseed)

image_path = "/datasets/nih-chest-xrays"

csv_file = pd.read_csv(os.path.join(image_path,"Data_Entry_2017.csv"))
test_split = os.path.join(image_path,"test_list.txt")
train_val_split = os.path.join(image_path,"train_val_list.txt")
with open(train_val_split, 'r') as f:
    train_val_images = f.read().splitlines()
with open(test_split, 'r') as f:
    test_images = f.read().splitlines()

train_df = csv_file[csv_file['Image Index'].isin(train_val_images)]

test_df = csv_file[csv_file['Image Index'].isin(test_images)]
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
train_ds = XrayDataset(train_df, image_path)
test_ds = XrayDataset(test_df, image_path)
train_prototype(args, train_ds,test_ds)

In [ ]:
# extracted from: https://captum.ai/tutorials/CIFAR_Captum_Robustness
classes = ('0','1','2','3','4','5','6','7','8','9','10','11','12','13','14')
def image_show(img, pred):
    npimg = img.squeeze().permute(1, 2, 0).cpu().detach().numpy()
    plt.imshow(npimg)
    plt.title("prediction: %s" % pred)
    plt.show()
    plt.savefig("counter_plot.png")


def get_prediction(model, input, normalize_im=False):
    if normalize:
      output = model(input)
      _, pred = torch.max(output, dim=1)
    return classes[pred], torch.sigmoid(output)

In [ ]:
# extracted from: https://captum.ai/tutorials/CIFAR_Captum_Robustness
def counterfactual_explanation(model, image,label):
    image = image.unsqueeze(0)
    normalize = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    image.requires_grad = True
    # Get original prediction
    pred, score  = get_prediction(model, image, normalize_im=False)
    preprocess = lambda ims: ims.to(image.device)

    feature_mask = torch.arange(64).reshape(8,8).repeat_interleave(repeats=28, dim=1).repeat_interleave(repeats=28, dim=0).reshape(1,1,224,224).to(image.device)
    ablator = FeatureAblation(model)

    active_labels = torch.nonzero(label.squeeze(), as_tuple=True)[0].to(image.device) # Get the indices of the active labels


    attr = ablator.attribute(image, target=active_labels.to(image.device), feature_mask=feature_mask.to(image.device))
    # Choose single channel, all channels have same attribution scores
    pixel_attr = attr[:,0:1].to(image.device)
    def pixel_dropout(image, dropout_pixels):

        keep_pixels = image[0][0].numel() - int(dropout_pixels)
        vals, _ = torch.kthvalue(pixel_attr.flatten(), keep_pixels)
        return (pixel_attr < vals.item()) * image

    min_pert_attr = MinParamPerturbation(forward_func=model.to(image.device), attack=pixel_dropout, arg_name="dropout_pixels", mode="linear",
                                     arg_min=0, arg_max=1024, arg_step=16,
                                     preproc_fn=preprocess, apply_before_preproc=True)
    pixel_dropout_im, pixels_dropped = min_pert_attr.evaluate(image, target=active_labels.to(image.device), perturbations_per_eval=10)
    # Feature Dropout Image
    new_pred_dropout, score_dropout = get_prediction(model, pixel_dropout_im, normalize_im=False)
    score_dropout_values = ", ".join(map(str, score_dropout.tolist()))
    image_show(pixel_dropout_im, new_pred_dropout)

In [ ]:
def train_baseline(args, train_ds,test_ds):



    batch_size=args.batch_size


    classifier_criterion = nn.BCEWithLogitsLoss()

    global f1_list,auc_list, precision_list, recall_list
    setup()
    init_fn = partial(
        worker_init_fn,
        num_workers=args.num_workers,
        rank=dist.get_rank(),
        seed=args.rseed,
    )

    test_sampler = DistributedSampler(test_ds, shuffle=True)

    test_dl = DataLoader(test_ds,sampler=test_sampler, batch_size=args.batch_size,worker_init_fn=init_fn)

    # setup()
    torch.cuda.set_device(int(os.environ["LOCAL_RANK"]))
    torch.cuda.empty_cache()
    device_id = torch.cuda.current_device()



    for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(train_ds)))):

        print("fold:", fold)

        print('Fold {}'.format(fold + 1))

        train_subset = Subset(train_ds, train_idx)
        val_subset = Subset(train_ds, val_idx)
        train_sampler = DistributedSampler(train_subset, shuffle=True)
        valid_sampler = DistributedSampler(val_subset, shuffle=True)
        init_fn = partial(worker_init_fn,num_workers=args.num_workers,rank=dist.get_rank(),seed=args.rseed)
        train_dl = DataLoader(train_ds, sampler=train_sampler,batch_size=args.batch_size,worker_init_fn=init_fn,pin_memory=False,drop_last=True,num_workers=args.num_workers)
        valid_dl = DataLoader(train_ds, sampler=valid_sampler, batch_size=args.batch_size,worker_init_fn=init_fn,pin_memory=False,drop_last=True,num_workers=args.num_workers)





        model2 = ResNetAttention(models.resnet18(pretrained=True))
        model2 = model2.cuda(device_id)
        model2 = DDP(model2, device_ids=[device_id])#,find_unused_parameters=True)
        optimizer = optim.AdamW(model2.parameters(), lr=args.lr,weight_decay=args.weight_decay)

        try:
            model2, optimizer, start_epoch, _ = load_checkpoint(model2, optimizer,"output_weight/resnet.pth")
            print(f"Resuming from epoch {start_epoch + 1}")
        except FileNotFoundError:
            print("No checkpoint found, starting frsom scratch")

        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, gamma=0.995, step_size=1)
        optimizer.zero_grad()
        cl_list=[]


        for epoch in range(args.num_epochs):
            if epoch<=20:
                continue
            model2.train()
            train_loss = 0
            counter = 0
            num_batches=0
            val_batch=0
            train_batches=0
            epoch_loss = 0

            train_loader_examples_num = len(train_dl.dataset)
            training_estimated = np.zeros((train_loader_examples_num, 15), dtype = np.float32)
            training_ture  = np.zeros((train_loader_examples_num, 15), dtype = np.float32)
            k=0
            for num_batches,(images,labels) in enumerate(train_dl):


                optimizer.zero_grad()
                images = images.cuda(device_id, non_blocking=True)
                counterfactual_explanation(model2, images[0],labels[0])
                labels = labels.cuda(device_id, non_blocking=True)


                logits= model2(images.float())
                prob = torch.sigmoid(logits)

                loss = classifier_criterion(logits, labels)
                heatmap = gradcam(model2,images,pred)
                model2.zero_grad()
                loss.backward()



                optimizer.step()
                if (num_batches+1)%8==0:
                    save_checkpoint(model2, optimizer, epoch, loss.item(), file_path=f"output_weight/resnet_backup.pth")

                save_checkpoint(model2, optimizer, epoch, loss.item(), file_path=f"output_weight/resnet.pth")

                training_estimated[k: k + prob.shape[0], :] = prob.detach().cpu().numpy()
                training_ture[k: k + prob.shape[0], :] = labels.detach().cpu().numpy()
                k += prob.shape[0]




            model2.eval()

            with torch.set_grad_enabled(False):

                val_loss = 0.0
                val_b=0
                total_epoch = 0

                val_loader_examples_num = len(valid_dl.dataset)
                validation_estimated = np.zeros((val_loader_examples_num, 15), dtype = np.float32)
                validation_true  = np.zeros((val_loader_examples_num, 15), dtype = np.float32)
                n = 0

                k = 0
                for val_batches,(images,labels) in enumerate(valid_dl):

                    labels =  labels.cuda(device_id, non_blocking=True)


                    images = images.cuda(device_id, non_blocking=True)
                    logits,attention_weights = model2(images)



                    prob = torch.sigmoid(logits)

                    validation_estimated[k: k + prob.shape[0], :] = prob.detach().cpu().numpy()
                    validation_true[k: k + prob.shape[0], :] = labels.detach().cpu().numpy()
                    k += prob.shape[0]

            val_auc = get_roc_auc_score(validation_true, validation_estimated)

            train_auc = get_roc_auc_score(training_ture, training_estimated)



            print("epoch",epoch,":","train_AUC:",train_auc,"val_AUC",val_auc)
            model2.eval()
        with torch.set_grad_enabled(False):


            test_loader_examples_num = len(test_dl.dataset)
            test_estimated = np.zeros((test_loader_examples_num, 15), dtype = np.float32)
            test_true  = np.zeros((test_loader_examples_num, 15), dtype = np.float32)
            n = 0

            k = 0

            all_predictions, all_labels = [], []
            for images,labels in test_dl:


                images, labels = images.cuda(device_id, non_blocking=True), labels.cuda(device_id, non_blocking=True)



                logits,attention_weights = model2(images)

                logits = torch.squeeze(logits)

                prob = torch.sigmoid(logits)

                threshold = 0.5
                pred =(prob >= threshold).float()
                print("this shape",attention_weights.shape,images.shape,pred.shape, attention_weights[0].shape,images[0].shape,pred[0].shape)
                visualize_trainable_attention(attention_weights[0],images[0],labels[0],pred[0])

                test_estimated[k: k + prob.shape[0], :] = prob.detach().cpu().numpy()
                test_true[k: k + prob.shape[0], :] = labels.detach().cpu().numpy()

                k += prob.shape[0]

                all_predictions = test_estimated
                all_labels = test_true


            final_preds, final_labels = aggregate_all_predictions(all_predictions, all_labels,device_id)

            # Calculate AUC on rank 0
            if dist.get_rank() == 0:
                auc = get_roc_auc_score( final_labels, final_preds)
                print(f"Test AUC: {auc}")


            break




    logging.info('Finished training.')

    dist.destroy_process_group()
    return 0

## References
[[1] Captum Robustness with Image Classification](https://captum.ai/tutorials/CIFAR_Captum_Robustness)